## 지번별 건물 에너지 사용량
+ 데이터명: 국토교통부 '건물에너지정보 서비스'
+ 데이터 유형: xml
+ 요청인자: 시군구코드, 법정동코드, 번, 지, 사용년월(YYYYMM)
+ 출력값: 사용량(useQty)

[공공데이터 링크](https://www.data.go.kr/data/15054212/openapi.do)

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

url = 'http://apis.data.go.kr/1611000/BldEngyService/getBeElctyUsgInfo'
params ={'serviceKey' : 'cTP7canYYa8SBQuAJzDFlO4fCg/krijsJXe3oW448oTBx8pj3b0rMaBzLpDN/AdIZim2GBbEuN1C6fIrSQ4K2w==', 'numOfRows' : '10', 'pageNo' : '1', 'sigunguCd' : '41290', 'bjdongCd' : '10700', 'bun' : '0101', 'ji' : '0000', 'useYm' : '202204' }

response = requests.get(url, params=params)
data = BeautifulSoup(response.text, 'lxml')
item = data.find_all("item")

In [ ]:
row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] # 데이터값

for i in range(0, len(item)):
    columns = item[i].find_all()
    for j in range(0,len(columns)):
        if i == 0:
            name_list.append(columns[j].name)
        value_list.append(columns[j].text)
    row_list.append(value_list)
    value_list=[]

name_list = ['법정동코드', '번', '지', '새주소본번', '새주소도로코드', '새주소부번', '도로명대지위치', '대지구분코드', '대지위치', '순번', '시군구코드', '사용량', '사용년월']

In [ ]:
print(name_list)

[]


## 단지별 에너지 사용량
+ 데이터명: 국토교통부 '공동주택 에너지 사용 정보'
+ 데이터 유형: xml
+ 요청인자: 서비스키, 단지코드, 발생년월(YYYYMM)
+ 출력값: 난방사용량(금액), 급탕사용량(금액), 가스사용량(금액), 전기사용량(금액), 수도사용량(금액)

[공공데이터 링크](https://www.data.go.kr/data/15012964/openapi.do)

### 단지코드 불러오기

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import time

In [ ]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# github 연결하기
%cd '/content/drive/MyDrive/국토교통데이터활용공모전'
!git clone https://github.com/haemilia/MolitContest.git

/content/drive/MyDrive/국토교통데이터활용공모전
Cloning into 'MolitContest'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 59 (delta 7), reused 21 (delta 2), pack-reused 0
Unpacking objects: 100% (59/59), 83.56 KiB | 161.00 KiB/s, done.


In [2]:
# AptList 폴더 내 자치구별 파일 리스트 불러오기
data_path = '../AptList'
file_list = os.listdir(data_path)

In [3]:
file_list

['Dobong_Gu.csv',
 'Dobong_more.csv',
 'Dongdaemun_Gu.csv',
 'Dongdaemun_more.csv',
 'Dongjak_Gu.csv',
 'Dongjak_more.csv',
 'Eunpyeong_Gu.csv',
 'Eunpyeong_more.csv',
 'Gangbuk_Gu.csv',
 'Gangbuk_more.csv',
 'Gangdong_Gu.csv',
 'Gangdong_more.csv',
 'Gangnam_Gu.csv',
 'Gangnam_more.csv',
 'Gangseo_Gu.csv',
 'Gangseo_more.csv',
 'Geumcheon_Gu.csv',
 'Geumcheon_more.csv',
 'Guro_Gu.csv',
 'Guro_more.csv',
 'Gwanak_Gu.csv',
 'Gwanak_more.csv',
 'GwangJin_Gu.csv',
 'Gwangjin_more.csv',
 'Jongno_Gu.csv',
 'Jongno_more.csv',
 'Jungnang_Gu.csv',
 'Jungnang_more.csv',
 'Jung_Gu.csv',
 'Jung_more.csv',
 'Mapo_Gu.csv',
 'Mapo_more.csv',
 'Nowon_Gu.csv',
 'Nowon_more.csv',
 'Seocho_Gu.csv',
 'Seocho_more.csv',
 'Seodaemun_Gu.csv',
 'Seodaemun_more.csv',
 'SeongBuk_Gu.csv',
 'Seongbuk_more.csv',
 'Seongdong_Gu.csv',
 'Seongdong_more.csv',
 'Songpa_Gu.csv',
 'Songpa_more.csv',
 'Yangcheon_Gu.csv',
 'Yangcheon_more.csv',
 'Yeongdeungpo_Gu.csv',
 'Yeongdeungpo_more.csv',
 'YongSan_Gu.csv',
 'Yongsan

In [4]:
# 구별 단지 코드 불러오는 함수 생성
def getDanjiCode(Gu):
  data_path = '../AptList/'
  gu_df = pd.read_csv(data_path + Gu, encoding = 'utf-8')
  danji_codes = gu_df['kaptCode']
  return danji_codes

In [ ]:
danji_codes = getDanjiCode(file_list[1])
energy_df = getEnergyUse(["A13789002"], "202209")

In [ ]:
energy_df

,code,date,elect,gas,heat,watercool,waterhot
0,A13789002,202209,37543,0,0,1819,0


### 단지별 난방, 급탕, 가스, 전기, 수도 사용량 불러오기

+ 우리가 비교하고 싶은 기간이 언제부터 언제까지인지 결정

In [ ]:
API_key = 'cTP7canYYa8SBQuAJzDFlO4fCg/krijsJXe3oW448oTBx8pj3b0rMaBzLpDN/AdIZim2GBbEuN1C6fIrSQ4K2w=='
url = 'http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService/getHsmpApHusUsgQtyInfoSearch'

params ={'serviceKey' : API_key,
              'kaptCode' : "A13203302",
              'reqDate' : "202201"}

response = requests.get(url, params)
data = BeautifulSoup(response.text, "xml")
data

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><item><elect>21531800</elect><gas>0</gas><heat>0</heat><helect>132451</helect><hgas>0</hgas><hheat>0</hheat><hwaterCool>7344</hwaterCool><hwaterHot>0</hwaterHot><kaptCode>A13203302</kaptCode><waterCool>6564380</waterCool><waterHot>0</waterHot></item></body></response>

In [35]:
def getEnergyUse(danji_codes, date):

  error = 0
  API_key = 'cTP7canYYa8SBQuAJzDFlO4fCg/krijsJXe3oW448oTBx8pj3b0rMaBzLpDN/AdIZim2GBbEuN1C6fIrSQ4K2w=='
  url = 'http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService/getHsmpApHusUsgQtyInfoSearch'

  energyUse_list = []

  for code in danji_codes:
    
    params ={'serviceKey' : API_key,
              'kaptCode' : code,
              'reqDate' : date}
    
    try: 
        response = requests.get(url, params)
        data = BeautifulSoup(response.text, "xml")
    
    except:
        time.sleep(2)

        response = requests.get(url, params)
        data = BeautifulSoup(response.text, "xml")

        
    if(data.find('helect') == None):
      error += 1
      continue
    elif(data.find('helect').get_text() == "0"):
      continue
    else:
      elect = data.find('helect').get_text()

    gas = data.find('hgas').get_text()

    heat = data.find('hheat').get_text()

    watercool = data.find('hwaterCool').get_text()

    waterhot = data.find('hwaterHot').get_text()

    data = {
        'code': code,
        'date': date,
        'elect': elect,
        'gas': gas,
        'heat': heat,
        'watercool': watercool,
        'waterhot': waterhot
    }

    energyUse_list.append(data)

  energy_df = pd.DataFrame(energyUse_list)
  print(error)

  return energy_df

### 3. 특정 기간 내 단지별 개별 에너지 사용량 csv로 저장


In [37]:
# 특정 기간 내 date list(YYYYMM) 생성
import datetime

start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2023, 4, 30)

date_list = []
current_date = start_date

while current_date <= end_date:
    date_string = current_date.strftime("%Y%m")
    date_list.append(date_string)
    current_date = current_date.replace(day=1) + datetime.timedelta(days=32)
    current_date = current_date.replace(day=1)

print(date_list)

['202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304']


In [16]:
# getDanjiCode로 특정 자치구의 단지 코드 불러오기
# for문과 getEnergyUse로 기간별 단지별 에너지 사용량 불러오기
def getGuEnergyDF(Gu_file):
  danji_codes = getDanjiCode(Gu_file)

  energy_df = pd.DataFrame()
  for date in date_list:
    df = getEnergyUse(danji_codes, date)
    energy_df = pd.concat([energy_df, df], ignore_index=True)

  save_path = f'../EnergyUseInfo/energy_{Gu_file}'

  if os.path.isfile(save_path):
    existing_df = pd.read_csv(save_path)
    energy_df = pd.concat([existing_df, energy_df], ignore_index=True)

    energy_df.to_csv(save_path, index=False)
  else:
    print("not existing")
    
  print(Gu_file)

In [39]:
def getGuEnergyDF(Gu_file):
  danji_codes = getDanjiCode(Gu_file)

  energy_df = pd.DataFrame()
  for date in date_list:
    df = getEnergyUse(danji_codes, date)
    energy_df = pd.concat([energy_df, df], ignore_index=True)

  save_path = f'../EnergyUseInfo/energy_{Gu_file}'

  energy_df.to_csv(save_path, index=False)

  print(Gu_file)

In [36]:
for gu in file_list[18::2]:
  getGuEnergyDF(gu)

142
132
134
135
135
150
133
135
138
147
135
128
Guro_Gu.csv
35
30
20
43
27
37
20
39
42
63
77
62
Gwanak_Gu.csv
81
66
81
72
72
81
68
81
79
69
81
69
GwangJin_Gu.csv
29
31
31
31
31
31
31
13
30
31
31
30
Jongno_Gu.csv
100
106
105
101
110
94
113
107
103
102
108
98
Jungnang_Gu.csv
46
46
46
43
31
46
46
46
46
42
34
46
Jung_Gu.csv
113
128
111
128
112
128
113
128
113
113
128
110
Mapo_Gu.csv
200
200
198
200
199
198
201
200
199
196
201
199
Nowon_Gu.csv
156
175
160
160
161
158
167
164
166
160
157
161
Seocho_Gu.csv
88
86
77
88
74
88
84
74
88
87
75
88
Seodaemun_Gu.csv
120
134
120
122
132
119
121
133
117
114
132
123
not existing
SeongBuk_Gu.csv
103
98
99
113
99
101
104
110
95
99
113
100
Seongdong_Gu.csv
138
133
138
139
139
134
138
140
138
137
138
138
Songpa_Gu.csv
116
132
118
118
117
126
123
116
116
115
117
115
Yangcheon_Gu.csv
150
150
145
142
144
148
148
150
150
150
133
95
Yeongdeungpo_Gu.csv
25
37
42
41
43
44
33
31
41
34
38
45
YongSan_Gu.csv


In [40]:
getGuEnergyDF(file_list[38])

66
70
66
73
59
68
67
70
66
66
62
64
66
67
76
71
56
69
76
65
73
74
66
56
65
61
70
76
SeongBuk_Gu.csv
